# 基于卷积神经网络的图像分类
该notebook共训练了五个 神经网络，分别为 **Densenet \ inceptionResnet v3 \ Inception  V3 \ Xception\ Resnet50**     
采用kaggle kernel +GPU的方式进行训练  

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))



['speech-train2', 'speech-test']


In [2]:

from keras.preprocessing import image 

from sklearn.model_selection import KFold
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import PReLU
from keras.optimizers import RMSprop,SGD
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, LearningRateScheduler
import keras.backend as K
import os
import sys
import glob
import matplotlib.pyplot as plt
import keras
from keras import __version__
from keras.applications.densenet import DenseNet201,preprocess_input
from keras.applications.xception import Xception,preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import preprocess_input
import numpy as np
import pandas as pd

Using TensorFlow backend.


## (1) 训练集读取

In [3]:
# 将图片地址存入 dataframe ， 以flow_from_dataframe 方式读取图片数据
train_df = pd.read_csv('../input/speech-train2/index.csv')
train_df['accent'] = train_df['accent'].astype('str')
print(type(train_df['accent'].values))
train_df['file_id'] = "../input/speech-train2/train/train/" +train_df['file_id']
train_df.head()

<class 'numpy.ndarray'>


,file_id,accent
0,../input/speech-train2/train/train/10000.png,1
1,../input/speech-train2/train/train/10001.png,1
2,../input/speech-train2/train/train/10002.png,0
3,../input/speech-train2/train/train/10003.png,2
4,../input/speech-train2/train/train/10004.png,1


In [4]:
IM_WIDTH, IM_HEIGHT = 128, 173 #densenet指定的图片尺寸

nb_classes = 3 # 分类数
batch_size = int(32)        
target_size = (128,173)
train_shape =  4500*0.85 # 训练样本个数3600
val_shape = 4500*0.15  # 验证样本个数


# 训练数据与测试数据

datagen = image.ImageDataGenerator(preprocessing_function=None,   #preprocess_input
                             rescale=1./255.,
                            zca_whitening = True,
#                               width_shift_range=0.1, 
#                                height_shift_range=0.1, 
                             validation_split=0.15)  #0.2
 
train_generator=datagen.flow_from_dataframe(
                        dataframe=train_df,
                        directory=None, 
                        x_col="file_id", 
                        y_col="accent",
                        has_ext=False,
                        subset="training",
                        batch_size=batch_size,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=target_size)  #(128, 173)


valid_generator=datagen.flow_from_dataframe(
                        dataframe=train_df,
                        directory=None,
                        x_col="file_id",
                        y_col="accent",
                        has_ext=False,
                        subset="validation",
                        batch_size=batch_size, #1
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=target_size)



/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 3825 validated image filenames belonging to 3 classes.
Found 675 validated image filenames belonging to 3 classes.


## (2) 测试集读取

In [5]:
# 测试集

folder_dir = "../input/speech-test/test/test/"  #测试集

test_df = pd.DataFrame(columns=['file_id','accent'])
i= 0 

for filename in os.listdir(folder_dir):
    file = folder_dir+filename
    test_df.at[i,'file_id'] = file
    i+=1

print(test_df.shape)

test_datagen=image.ImageDataGenerator(rescale=1./255.,
                                     preprocessing_function=None,   #preprocess_input
                             )

test_generator=test_datagen.flow_from_dataframe(
                            dataframe=test_df,
                            directory=None,
                            x_col="file_id",
                            y_col=None,
                            has_ext=False,
                            batch_size=batch_size,#1
                            seed=42,
                            shuffle=False,
                            class_mode=None,
                            target_size=target_size)
test_df.head()

(5377, 2)
Found 5377 validated image filenames.


,file_id,accent
0,../input/speech-test/test/test/21352.png,NaN
1,../input/speech-test/test/test/21121.png,NaN
2,../input/speech-test/test/test/23559.png,NaN
3,../input/speech-test/test/test/20365.png,NaN
4,../input/speech-test/test/test/24472.png,NaN


## （3）inception v3测试

In [6]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception,preprocess_input

### 1. 模型训练

In [7]:
def model_training(base_model,output_model_file):
    nb_classes = 3 # 分类数
    nb_epoch = int(30)                # epoch数量

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predicts =Dense(3, activation='softmax')(x) # for i in range(3) ] 


    v3_model = Model(inputs=base_model.input, outputs=predicts)  #
    v3_model.compile(
                    optimizer='adam',

                    loss='categorical_crossentropy', metrics=['accuracy'])


    v3_output_model_file =output_model_file #'v3_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'

    v3_checkpoint = ModelCheckpoint(v3_output_model_file, monitor='val_acc',
                                 verbose=1, save_best_only=True)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                                    patience=4,
                                                    verbose=2,
                                                    factor=.3,
                                                    min_lr=.00001)

    v3_history_ft = v3_model.fit_generator(
                                    train_generator,
                                    nb_epoch=nb_epoch,  #
                                    validation_data=valid_generator,
                                    validation_steps=val_shape // batch_size,

                                    steps_per_epoch=train_shape // batch_size,
                                    callbacks=[learning_rate_reduction,v3_checkpoint], #
    )

In [8]:
# 模型训练
# dense_model = DenseNet201(include_top=False)
# model_training( dense_model , output_model_file = 'dense_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5')

# v3_model = InceptionV3(include_top=False ) 
# model_training( v3_model , output_model_file = 'v3_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5')

# x_model = Xception(include_top=False )
# model_training( x_model , output_model_file = 'v3_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5')

v2_model = InceptionResNetV2(include_top=False )
model_training( v2_model , output_model_file = 'dense_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5')

# v2_model = ResNet50(include_top=False )
# model_training( v2_model , output_model_file = 'dense_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5')

219062272/219055592 [==============================] - 8s 0us/step


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., validation_steps=21.0, steps_per_epoch=119.0, callbacks=[<keras.ca..., epochs=30)`


Epoch 1/30


/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


119/119 [==============================] - 90s 754ms/step - loss: 1.0073 - acc: 0.5454 - val_loss: 1.9908 - val_acc: 0.4851

Epoch 00001: val_acc improved from -inf to 0.48512, saving model to dense_checkpoint-01e-val_acc_0.49.hdf5
Epoch 2/30
119/119 [==============================] - 41s 347ms/step - loss: 0.7709 - acc: 0.6809 - val_loss: 1.7794 - val_acc: 0.3624

Epoch 00002: val_acc did not improve from 0.48512
Epoch 3/30
119/119 [==============================] - 41s 341ms/step - loss: 0.6440 - acc: 0.7453 - val_loss: 1.6223 - val_acc: 0.5863

Epoch 00003: val_acc improved from 0.48512 to 0.58631, saving model to dense_checkpoint-03e-val_acc_0.59.hdf5
Epoch 4/30
119/119 [==============================] - 41s 345ms/step - loss: 0.5442 - acc: 0.7978 - val_loss: 4.9937 - val_acc: 0.4603

Epoch 00004: val_acc did not improve from 0.58631
Epoch 5/30
119/119 [==============================] - 40s 339ms/step - loss: 0.4714 - acc: 0.8283 - val_loss: 8.2243 - val_acc: 0.3561

Epoch 00005: v

### 2. 模型存储

In [9]:

files = os.listdir("../working")
print(files)
print()
v3_checkpoint = []
dense_checkpoint = []
for file in files:

    if file.startswith('v3_checkpoint'):  # multi checkpoint
        v3_checkpoint.append(file)
    if file.startswith('dense_checkpoint'):  # multi checkpoint
        dense_checkpoint.append(file)
        
v3_checkpoint = sorted(v3_checkpoint)      
dense_checkpoint = sorted(dense_checkpoint)
print((v3_checkpoint))
print(dense_checkpoint)

['dense_checkpoint-03e-val_acc_0.59.hdf5', 'dense_checkpoint-06e-val_acc_0.72.hdf5', 'dense_checkpoint-11e-val_acc_0.78.hdf5', 'dense_checkpoint-01e-val_acc_0.49.hdf5', '__notebook__.ipynb', '__output__.json', 'dense_checkpoint-14e-val_acc_0.80.hdf5']

[]
['dense_checkpoint-01e-val_acc_0.49.hdf5', 'dense_checkpoint-03e-val_acc_0.59.hdf5', 'dense_checkpoint-06e-val_acc_0.72.hdf5', 'dense_checkpoint-11e-val_acc_0.78.hdf5', 'dense_checkpoint-14e-val_acc_0.80.hdf5']


### 3. 模型预测

In [10]:
def output(model , test_generator, name):
    # 模型预测

    pred = model.predict_generator(test_generator,steps= len(test_generator)) 
    
    df = pd.DataFrame(columns=['file_id','accent'])  #输出文件
    i= 0

    predicted_class_indices = np.argmax(pred, axis=1)

    
    # # #建立预测结果和文件名之间的关系
    filenames = test_generator.filenames
    for idx in range(len(filenames )):
        df.at[i,'file_id'] = filenames[idx]
        df.at[i,'accent'] = (int(predicted_class_indices[idx]))
        i += 1
#         print('predict  %d' % (int(predicted_class_indices[idx])),'title    %s' % filenames[idx])


    df['file_id'] = df['file_id'].apply(lambda x: x.split('/')[-1].split('.')[0])
    df = df.sort_values(by = 'file_id' , ascending = True)
    print(df.head())
    df.to_csv(name,index = False)
    return df


In [11]:
# v3 模型输出
# v3_model = load_model(v3_checkpoint[-1])
# df = output(v3_model ,test_generator,'x_submission.csv')  # 利用已加载的
# print(df.accent.value_counts())

#densenet 模型输出
dense_model = load_model(dense_checkpoint[-1])
df = output(dense_model ,test_generator,'v2_submission.csv')  # 利用已加载的
print(df.accent.value_counts())

     file_id accent
1753   20000      1
2831   20001      1
2603   20002      1
1355   20003      0
3850   20004      2
0    2129
1    1628
2    1620
Name: accent, dtype: int64


## (3) bottleneck vgg16提取特征  
经试验验证，发现 此方法效果并不好，因此不采用

In [12]:
'''
img_width, img_height = 173, 128

class_indics = 'class_indices.npy'
bottleneck_train_path = 'bottleneck_features_train.npy'
bottleneck_validation_path = 'bottleneck_features_validation.npy'
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation/'

nb_train_samples = 3600
nb_validation_samples = 900

epochs = 20
batch_size = 32
'''

"\nimg_width, img_height = 173, 128\n\nclass_indics = 'class_indices.npy'\nbottleneck_train_path = 'bottleneck_features_train.npy'\nbottleneck_validation_path = 'bottleneck_features_validation.npy'\ntop_model_weights_path = 'bottleneck_fc_model.h5'\ntrain_data_dir = 'data/train'\nvalidation_data_dir = 'data/validation/'\n\nnb_train_samples = 3600\nnb_validation_samples = 900\n\nepochs = 20\nbatch_size = 32\n"

In [13]:
##  获取generator 标签
index = pd.read_csv('../input/speech-train2/index.csv')

def get_labels(generator,index, ):
    labels = []
    for i in generator.filenames:
        name = i.split('/')[-1]
        label = index[index['file_id']==name].accent.values[0]
#         print( name ,label)
        labels.append(label)
    
    labels = keras.utils.to_categorical(labels,num_classes=3)
    return labels


In [14]:
def generate_class_indics():
    datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator=datagen.flow_from_dataframe(
                        dataframe=train_df,
                        directory=None, 
                        x_col="file_id", 
                        y_col="accent",
                        has_ext=False,
                        batch_size=batch_size,
                        seed=42,
                        class_mode="categorical",
                        target_size=(128, 173))  #(128, 173)

    # save the class indices to use later in predictions
    np.save(class_indics, train_generator.class_indices)

In [15]:
def save_bottleneck_features(train_generator,valid_generator):
    print('Using of bottleneck feature on pretrained model started.')
    
    # build the VGG16 network
#     model = VGG16(include_top=False, weights='imagenet')
    model = Xception(include_top=False)

    
    bottleneck_features_train = model.predict_generator( train_generator, 
                                                        steps = len(train_generator)  #nb_train_samples // batch_size
                                                       )
    
    np.save(open(bottleneck_train_path, 'wb'), bottleneck_features_train)
    print('train bottleneck shape',bottleneck_features_train.shape)

    bottleneck_features_validation = model.predict_generator(valid_generator,
                                                             steps = len(valid_generator)#nb_validation_samples // batch_size
                                                            )
    np.save(open(bottleneck_validation_path, 'wb'), bottleneck_features_validation)
    print('validation bottleneck shape',bottleneck_features_validation.shape)
    print('Using of bottleneck feature on pretrained model finished.')


In [16]:

def train_top_model():
    print('Training of top model started.')
    
    train_data = np.load(open(bottleneck_train_path, 'rb'))
    validation_data = np.load(open(bottleneck_validation_path, 'rb'))


    train_labels = get_labels(train_generator,index )
    val_labels = get_labels(valid_generator,index)
    
    class_dictionary = np.load('class_indices.npy', allow_pickle=True).item()
    num_classes = len(class_dictionary)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax')) #sigmoid

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])  #categorical_

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, val_labels))
    
    model.save_weights(top_model_weights_path)
    print('Training of top model completed & saved as: ',top_model_weights_path)


def fine_tune_pretrained_model():
    print('Fine tuning of pretrain model started.')
    # build the VGG16 network
    input_tensor = Input(shape=(128, 173, 3))

#     base_model = applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
#     base_model = applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
    base_model = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)


    class_dictionary = np.load('class_indices.npy', allow_pickle=True).item()
    num_classes = len(class_dictionary)

    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(1024, activation='relu'))
    top_model.add(Dropout(0.3))
    top_model.add(Dense(num_classes, activation='softmax')) #sigmoid

    # note that it is necessary to start with a fully-trained
    # classifier, including the top classifier,
    # in order to successfully do fine-tuning
    top_model.load_weights(top_model_weights_path)

    # add the model on top of the convolutional base
    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers[:25]:
        layer.trainable = False

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(loss='categorical_crossentropy',
#                   optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  optimizer = 'adam',
                  metrics=['accuracy'])  #categorical_

    # prepare data augmentation configuration
#     train_datagen = ImageDataGenerator(
#         rescale=1. / 255,
        
#         horizontal_flip=True)

#     test_datagen = ImageDataGenerator(rescale=1. / 255)

#     train_generator = train_datagen.flow_from_directory(
#         train_data_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         class_mode='categorical')

#     validation_generator = test_datagen.flow_from_directory(
#         validation_data_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         class_mode='categorical')

#     # fine-tune the model
#     model.fit_generator(
#         train_generator,
#         steps_per_epoch=nb_train_samples // batch_size, # samples_per_epoch=nb_train_samples,
#         epochs=epochs,
#         validation_data=validation_generator,
#         validation_steps=nb_validation_samples)

#     print('Fine tuning of pretrain model completed.')


In [17]:
print(os.listdir('../working'))

['v2_submission.csv', 'dense_checkpoint-03e-val_acc_0.59.hdf5', 'dense_checkpoint-06e-val_acc_0.72.hdf5', 'dense_checkpoint-11e-val_acc_0.78.hdf5', 'dense_checkpoint-01e-val_acc_0.49.hdf5', '__notebook__.ipynb', '__output__.json', 'dense_checkpoint-14e-val_acc_0.80.hdf5']


In [18]:
'''
if not os.path.exists(class_indics):
    generate_class_indics()

save_bottleneck_features(train_generator , valid_generator)

train_top_model()
fine_tune_pretrained_model()
'''

'\nif not os.path.exists(class_indics):\n    generate_class_indics()\n\nsave_bottleneck_features(train_generator , valid_generator)\n\ntrain_top_model()\nfine_tune_pretrained_model()\n'